# Exploration of Data.gov.sg's taxi and carpark availability API

Ref: https://beta.data.gov.sg/collections/85/view (carpark) ; https://beta.data.gov.sg/collections/352/view (taxi availability)

Data of interest: Realtime carpark and taxi availability.

## Notes:
- Carpark data only contains carpark ID without lat/lon location. A separate mapping file is required to facilitate location identification.
- For taxi availability, the coordinates and the total count are available.

In [3]:
from random import randint
from typing import Union, Dict
import requests

### Helper functions

In [9]:
def api_query(api_link: str,  agent_id: str) -> Union[Dict,None]:
    """Function which executes query via an api link using a provided agent_id as an identifier to avoid rejection of query request

    Args:
        api_link (str): API Link which requests is to be made
        agent_id (str): Id used for request header

    Returns:
        Dictioanry containing request content. None when exception are encountered.
    """
    req_headers = {"User-agent": agent_id}
    try:
        res = requests.get(api_link,
                            headers=req_headers,
                            timeout=5)
        # Raise if HTTPError occured
        res.raise_for_status()

        # Check the status code before extending the number of posts
        if res.status_code == 200:
            print(f"Request sucessful with {res.status_code}")
            the_json = res.json()
        return the_json

    except requests.exceptions.HTTPError as errh:
        print(errh)
    except requests.exceptions.ConnectionError as errc:
        print(errc)
    except requests.exceptions.Timeout as errt:
        print(errt)
    except requests.exceptions.RequestException as err:
        print(err)
    return None

def query_taxi_availability(api_link: str = "https://api.data.gov.sg/v1//transport/taxi-availability", agent_id: str= "test_qzq"): 
    taxi_avail_response = api_query(api_link=api_link, agent_id=agent_id)
    if not taxi_avail_response:
        print("No response received")
        return {}
    else:
        #print(taxi_avail_response)
        taxi_avail_response_features = taxi_avail_response["features"][0]
        geo_coord = taxi_avail_response_features["geometry"]["coordinates"]
        print(f"Number of coordinates received {len(geo_coord)}")
        taxi_counts_dict = taxi_avail_response_features["properties"]
        print(geo_coord)
        print(taxi_counts_dict)
    return None



def query_carpark_availability(api_link: str = "https://api.data.gov.sg/v1/transport/carpark-availability", agent_id: str= "test_qzq"): 
    carpark_avail_response = api_query(api_link=api_link, agent_id=agent_id)

    carpark_dict = {}
    if not carpark_avail_response:
        print("No response received")
        
    else:
        carpark_data_list = carpark_avail_response["items"][0]["carpark_data"]
        print(len(carpark_data_list))
        for carpark_info_dict in carpark_data_list:
            carpark_number = carpark_info_dict.pop("carpark_number")
            carpark_capacity_info = carpark_info_dict["carpark_info"][0]
            if carpark_number not in carpark_dict:
                carpark_dict[carpark_number] = carpark_capacity_info
            else:
                carpark_dict[carpark_number].update(carpark_capacity_info)
    return carpark_dict

### Function call to explore response outputs

In [5]:
query_taxi_availability()

Request sucessful with 200
Number of coordinates received 1505
[[103.62706, 1.2746], [103.63046, 1.31925], [103.64211, 1.33422], [103.64384, 1.32906], [103.64533, 1.33448], [103.66239, 1.3159], [103.67884, 1.31927], [103.68131, 1.34168], [103.6856115, 1.34087033333333], [103.68582, 1.3465], [103.6886, 1.34513], [103.68895, 1.34], [103.6891, 1.34], [103.69041, 1.34], [103.69283, 1.34626], [103.69351, 1.34], [103.69415, 1.37], [103.6949, 1.3247], [103.69627, 1.34], [103.696273616667, 1.3151716], [103.696884766667, 1.35026281666667], [103.696974216667, 1.3508636], [103.6970517, 1.34364643333333], [103.697282916667, 1.34737415], [103.69732, 1.35], [103.699184716667, 1.33903818333333], [103.6992, 1.34], [103.69921, 1.34], [103.69931, 1.34], [103.700222166667, 1.3519864], [103.70324, 1.34404], [103.703613416667, 1.3405559], [103.70392, 1.35], [103.70424385, 1.33847123333333], [103.704382133333, 1.34187213333333], [103.70595, 1.42755], [103.70631, 1.3367], [103.708285966667, 1.36285523333333]

In [10]:
query_carpark_availability()

Request sucessful with 200
1946


{'HE12': {'total_lots': '105', 'lot_type': 'C', 'lots_available': '0'},
 'HLM': {'total_lots': '583', 'lot_type': 'C', 'lots_available': '2'},
 'RHM': {'total_lots': '329', 'lot_type': 'C', 'lots_available': '172'},
 'BM29': {'total_lots': '97', 'lot_type': 'C', 'lots_available': '0'},
 'Q81': {'total_lots': '96', 'lot_type': 'C', 'lots_available': '8'},
 'C20': {'total_lots': '176', 'lot_type': 'C', 'lots_available': '1'},
 'FR3M': {'total_lots': '228', 'lot_type': 'C', 'lots_available': '102'},
 'C32': {'total_lots': '289', 'lot_type': 'C', 'lots_available': '231'},
 'C6': {'total_lots': '332', 'lot_type': 'C', 'lots_available': '108'},
 'TG2': {'total_lots': '273', 'lot_type': 'C', 'lots_available': '109'},
 'BP1': {'total_lots': '577', 'lot_type': 'C', 'lots_available': '373'},
 'TG1': {'total_lots': '133', 'lot_type': 'C', 'lots_available': '101'},
 'TGM2': {'total_lots': '189', 'lot_type': 'C', 'lots_available': '129'},
 'TE14': {'total_lots': '134', 'lot_type': 'C', 'lots_availa